Code for Model finetuning on labelled dataset.
Finetuning pre-trained "distilbert-base-uncased" on combined_dataset for sentiment analysis.
Finetuning pre-trained KeyBART on combined_dataset for Trends Extractions.

In [ ]:
!pip install transformers datasets scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


Here we are extracting sentences, sentiments, keywords and trends from labelled dataset and preparing training and testing data.

In [ ]:
import json
import random
from sklearn.model_selection import train_test_split

# Load your labeled dataset
dataset_path = "/content/combined_dataset.json"
with open(dataset_path, "r") as file:
    data = json.load(file)

# Extract sentences, sentiments, and trends
sentences = [item["sentence"] for item in data]
sentiments = [item["sentiment"] for item in data]
keywords = [item["keywords"] for item in data]
trends = [item["trends"] for item in data]

# Flatten sentences 
flat_sentences = [" ".join(sentence_group) for sentence_group in sentences]

# Split into train and test
train_texts, test_texts, train_sentiments, test_sentiments, train_trends, test_trends = train_test_split(
    flat_sentences, sentiments, trends, test_size=0.2, random_state=42
)


Code for Finetuning DistilBERT for sentiment analysis on training dataset.

In [ ]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch

# Tokenization
sentiment_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Prepare datasets
train_dataset = Dataset.from_dict({
    "text": train_texts,
    "labels": [1 if sentiment == "positive" else 0 for sentiment in train_sentiments]
})
test_dataset = Dataset.from_dict({
    "text": test_texts,
    "labels": [1 if sentiment == "positive" else 0 for sentiment in test_sentiments]
})

def tokenize_function(examples):
    return sentiment_tokenizer(examples["text"], truncation=True, padding=True, max_length=256)

train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Model setup
sentiment_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01
)

# Trainer
trainer = Trainer(
    model=sentiment_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=sentiment_tokenizer
)

# Fine-tune the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.421600,0.324980
2,0.267200,0.399453
3,0.129600,0.672810
4,0.057900,0.703585
5,0.026200,0.710535


TrainOutput(global_step=2855, training_loss=0.16015489347761858, metrics={'train_runtime': 651.4273, 'train_samples_per_second': 35.031, 'train_steps_per_second': 4.383, 'total_flos': 1511326760675616.0, 'train_loss': 0.16015489347761858, 'epoch': 5.0})

Evaluating model on testing dataset and printing evaluation metrics

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

# Predictions
predictions = trainer.predict(test_dataset)
preds = torch.argmax(torch.tensor(predictions.predictions), axis=1).numpy()

# Calculate metrics
accuracy = accuracy_score(test_dataset["labels"], preds)
precision, recall, f1, _ = precision_recall_fscore_support(test_dataset["labels"], preds, average="binary")

print(f"Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")


Accuracy: 0.8729, Precision: 0.9003, Recall: 0.8913, F1 Score: 0.8958


Now finetuning KeyBART model on testing data

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer, TrainingArguments, Trainer
from datasets import Dataset
import torch

# Load tokenizer and model
trends_tokenizer = BartTokenizer.from_pretrained("ml6team/keyphrase-generation-keybart-inspec")
trends_model = BartForConditionalGeneration.from_pretrained("ml6team/keyphrase-generation-keybart-inspec")

def preprocess_trends_data(texts, targets):
    inputs = [f"extract keyphrases: {text}" for text in texts]
    # Tokenize and convert to lists for Dataset.from_dict
    input_encodings = trends_tokenizer(inputs, max_length=512, truncation=True, padding=True)
    target_encodings = trends_tokenizer(targets, max_length=512, truncation=True, padding=True)
    return {
        "input_ids": input_encodings.input_ids,
        "attention_mask": input_encodings.attention_mask,
        "labels": target_encodings.input_ids,
    }

# Preprocess data
train_data = preprocess_trends_data(train_texts, ["; ".join(trend) for trend in train_trends])
test_data = preprocess_trends_data(test_texts, ["; ".join(trend) for trend in test_trends])

# Create datasets using Dataset.from_dict
train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)

# Fine-tuning
training_args = TrainingArguments(
    output_dir="./trends_results",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=trends_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    processing_class=trends_tokenizer
)

# Fine-tune
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-8-cad2637d8a41>:38: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,0.844000,0.942207
2,0.609500,0.888445
3,0.467600,0.869039


/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2817: UserWarning: Moving the following attributes in the config to the generation config: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=3423, training_loss=0.7979888782283907, metrics={'train_runtime': 3565.2682, 'train_samples_per_second': 3.84, 'train_steps_per_second': 0.96, 'total_flos': 1.4835998106058752e+16, 'train_loss': 0.7979888782283907, 'epoch': 3.0})

Here we are printing few samples from test dataset along with predicted trends by finetuned model and actual trends present in labelled dataset.
We are saving the output printed in "finteuned_samples_generated_trends.txt" file, to get evaluation metrics.

In [ ]:

%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

# Generate predictions
test_input_ids = test_dataset["input_ids"]  # Get input_ids from test_dataset
batch_size = 4  
predicted_trends = []
for i in range(0, len(test_input_ids), batch_size):
    # Convert the list of lists to a PyTorch tensor
    batch_input_ids = torch.tensor(test_input_ids[i : i + batch_size])
    # Now move it to the device
    batch_input_ids = batch_input_ids.to(trends_model.device)
    outputs = trends_model.generate(
        batch_input_ids, max_length=50, num_beams=5, early_stopping=True
    )
    predicted_trends.extend(
        [trends_tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    )

# Compare predicted vs actual
for i, (pred, actual) in enumerate(zip(predicted_trends, test_trends)):
    print(f"Sample {i}:")
    print(f"  Predicted: {pred}")
    print(f"  Actual: {'; '.join(actual)}")
    print("-" * 20)


env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
Sample 0:
  Predicted: collection ;  center ;  brilliant ;  one ;  diamonds
  Actual: center; collection; come; diamonds; brilliance; one; spinning
--------------------
Sample 1:
  Predicted: yet ;  cut ;  start ;  nine ;  source ;  diamonds ;  major
  Actual: nine; cut; yet; start; underground; finished; red
--------------------
Sample 2:
  Predicted: controlled ;  de ;  beers ;  historically ;  important ;  look ;  pricing
  Actual: controlled; primarily; diamond; organization; de; historically; important
--------------------
Sample 3:
  Predicted: things ;  working ;  always ;  us ;  know ;  take ;  complex
  Actual: things; always; working; us; take; easy; hone
--------------------
Sample 4:
  Predicted: know ;  stores ;  united ;  year ;  million ;  selling ;  hundred
  Actual: know; stores; working; united; top; year; selling
--------------------
Sample 5:
  Predicted: rich ;  world ;  hurting ;  actually ;  got ;  answer ;  b

One Sample text is passed to perform sentiment analysis and trends extraction by finetuned model.

In [ ]:
def analyze_text(input_text):
    # Sentiment analysis
    sentiment_inputs = sentiment_tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)
    sentiment_outputs = sentiment_model(**sentiment_inputs)
    sentiment = "positive" if torch.argmax(sentiment_outputs.logits) == 1 else "negative"

    # Trends extraction
    trends_inputs = trends_tokenizer(f"extract keyphrases: {input_text}", return_tensors="pt", truncation=True, padding=True)

    # Move input tensors to the same device as the model
    trends_inputs = trends_inputs.to(trends_model.device)
    trends_outputs = trends_model.generate(trends_inputs.input_ids, max_length=50, num_beams=5, early_stopping=True)
    trends = trends_tokenizer.decode(trends_outputs[0], skip_special_tokens=True)

    return sentiment, trends

# Test combined model
test_sentence = "Lab-grown diamonds are revolutionizing the diamond industry with ethical practices."
result_sentiment, result_trends = analyze_text(test_sentence)

print(f"Sentiment: {result_sentiment}")
print(f"Trends: {result_trends}")


Sentiment: positive
Trends: diamond ;  practices ;  industry ;  revolutionizing ;  ethical ;  lab-grown


The model is evaluated on different metrics based on predicted and actual trends saved in finteuned_samples_generated_trends.txt file.

In [ ]:
#!pip install rouge-score

from sklearn.metrics import precision_score, recall_score, f1_score
from rouge_score import rouge_scorer

# File path
file_path = "/content/finteuned_samples_generated_trends.txt"

# Initialize variables
predicted_list = []
actual_list = []

# Parse the file
with open(file_path, "r") as f:
    for line in f:
        if line.startswith("  Predicted:"):
            predicted = line.strip().split(":")[1].strip().split("; ")
            predicted_list.append(predicted)
        elif line.startswith("  Actual:"):
            actual = line.strip().split(":")[1].strip().split("; ")
            actual_list.append(actual)

def mean_reciprocal_rank(predicted, actual):
    mrr_scores = []
    for pred, act in zip(predicted, actual):
        ranks = [i + 1 for i, p in enumerate(pred) if p in act]
        mrr_scores.append(1 / ranks[0] if ranks else 0)
    return sum(mrr_scores) / len(mrr_scores)

def precision_at_k(predicted, actual, k=5):
    precision_scores = []
    for pred, act in zip(predicted, actual):
        pred_top_k = pred[:k]
        tp = sum(1 for p in pred_top_k if p in act)
        precision_scores.append(tp / k)
    return sum(precision_scores) / len(precision_scores)

def recall_at_k(predicted, actual, k=5):
    recall_scores = []
    for pred, act in zip(predicted, actual):
        pred_top_k = pred[:k]
        tp = sum(1 for p in pred_top_k if p in act)
        recall_scores.append(tp / len(act) if act else 0)
    return sum(recall_scores) / len(recall_scores)

def f1_at_k(predicted, actual, k=5):
    precision = precision_at_k(predicted, actual, k)
    recall = recall_at_k(predicted, actual, k)
    if precision + recall == 0:
        return 0
    return 2 * (precision * recall) / (precision + recall)

# Helper function to flatten and get unique elements
def flatten_and_unique(lst):
    return list(set([item.strip().lower() for sublist in lst for item in sublist]))

# Flatten and deduplicate
flat_predicted = flatten_and_unique(predicted_list)
flat_actual = flatten_and_unique(actual_list)

# Compute ROUGE metrics
rouge_scorer_obj = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = [rouge_scorer_obj.score(" ".join(pred), " ".join(act))
                for pred, act in zip(predicted_list, actual_list)]

# Compute average ROUGE scores
avg_rouge1 = sum([score['rouge1'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rouge2 = sum([score['rouge2'].fmeasure for score in rouge_scores]) / len(rouge_scores)
avg_rougeL = sum([score['rougeL'].fmeasure for score in rouge_scores]) / len(rouge_scores)

print(f"Length of y_pred: {len(flat_predicted)}")
print(f"Length of y_true: {len(flat_actual)}")

min_length = min(len(flat_predicted), len(flat_actual))
flat_predicted = flat_predicted[:min_length]
flat_actual = flat_actual[:min_length]

# Evaluate Metrics
mrr = mean_reciprocal_rank(flat_predicted, flat_actual)
precision_k = precision_at_k(flat_predicted, flat_actual, k=5)
recall_k = recall_at_k(flat_predicted, flat_actual, k=5)
f1_k = f1_at_k(flat_predicted, flat_actual, k=5)

# Display metrics
print(f"Mean Reciprocal Rank (MRR): {mrr:.4f}")
print(f"Precision@k: {precision_k:.4f}")
print(f"Recall@k: {recall_k:.4f}")
print(f"F1@k: {f1_k:.4f}")
print(f"Average ROUGE-1: {avg_rouge1:.4f}")
print(f"Average ROUGE-2: {avg_rouge2:.4f}")
print(f"Average ROUGE-L: {avg_rougeL:.4f}")


Length of y_pred: 2048
Length of y_true: 2431
Mean Reciprocal Rank (MRR): 0.5128
Precision@k: 0.3332
Recall@k: 0.2495
F1@k: 0.2854
Average ROUGE-1: 0.5835
Average ROUGE-2: 0.1395
Average ROUGE-L: 0.4343
